In [1]:
import telegram
import pickle
import yaml
from googlesearch import search

from telegram import (
    Poll,
    ParseMode,
    KeyboardButton,
    KeyboardButtonPollType,
    ReplyKeyboardMarkup,
    ReplyKeyboardRemove,
    Update,
    Bot,
)

from telegram.ext import (
    Updater,
    CommandHandler,
    PollAnswerHandler,
    PollHandler,
    MessageHandler,
    Filters,
    CallbackContext,
    Handler,
    CallbackQueryHandler,
    PollHandler,
)

import logging

from Recommender import (
    Recommender,
    Preprocessor,
    LDAu,
    User
)

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO,
)

/home/albert/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Get QA dictionary:

In [2]:
with open('questionnaire.yaml') as f:
    survey_dict = yaml.safe_load(f)

In [3]:
survey_dict['q1'][0][0]

"Assuming you'd never gain a kilo, if you could only eat one of these chocolates for the rest of your life, which would it be?:"

In [4]:
survey_dict['q1'][1]

['White chocolate', 'Milk chocolate', 'Dark chocolate']

## Get Recommender:

In [5]:
n_topics = 14 

with open('recommender.pickle', 'rb') as f:
     advisor = pickle.load(f)

## Create a bot:

In [6]:
def start(update: Update, context: CallbackContext) -> None:
    """Inform user about what this bot can do"""
    
    print("In question ...")
    update.message.reply_text(
        'Please select /poll to get a Poll questionnaire about wine preferences'
    )

def poll(update: Update, context: CallbackContext) -> None:
    """Sends a predefined poll"""
    
    
    arr_q = list(survey_dict.keys()) 
    array_of_answers = [0] * len(arr_q)
    
    print(arr_q, array_of_answers)
    
    message = context.bot.send_poll(
        update.effective_chat.id,
        context.bot_data['q1'][0][0],
        context.bot_data['q1'][1],
        is_anonymous=False,
        allows_multiple_answers=False,
    )
    
    payload = {
        message.poll.id: {
            "questions": survey_dict['q1'][1],
            "message_id": message.message_id,
            "chat_id": update.effective_chat.id,
            "answers": 0,
            "count": 1,
            "arr_q": arr_q,
            "array_of_answers": array_of_answers,
        }
    }
    context.bot_data.update(payload)

    
def receive_poll_answer(update: Update, context: CallbackContext) -> None:
    """Summarize a users poll vote"""
    
    answer = update.poll_answer
    poll_id = answer.poll_id
    
    num_calls = context.bot_data[poll_id]['count']
    print(f"\nNUM CALLS!!!:{num_calls}\n")
    
    len_arr_q = len(context.bot_data[poll_id]['arr_q'])
    if num_calls < len_arr_q:
        context.bot_data[poll_id]['array_of_answers'][num_calls - 1] = answer.option_ids
    
    else:
        context.bot_data[poll_id]['array_of_answers'][num_calls - 1] = answer.option_ids
        
        text = [survey_dict[context.bot_data[poll_id]['arr_q'][i]][1]\
                [context.bot_data[poll_id]['array_of_answers'][i][0]] for i in range(len_arr_q)]
        
        print(text)
        
        user = User(
            taste=" ".join(text),
            country=None,
            price_range=None,
            points_range=None,
        )
        
        top = 5
        top_wine, ranking, lda_u, tfidf_u = advisor.recommend(
            query=user, top=top, project=False,
        )
        
        context.bot.send_message(
            chat_id=context.bot_data[poll_id]["chat_id"],
            text="Let's get some wine Recommendations:\n"
        )
        
        context.bot.send_message(
            chat_id=context.bot_data[poll_id]["chat_id"],
            text=text
        )
        
        for wine_ind in range(top):
            query = top_wine['title'].iloc[wine_ind]
            cnt = 0
            for result in search(query, tld="com", num=10, stop=10, pause=0.01):
                cnt += 1
                if "vivino" in result:
                    context.bot.send_message(
                        chat_id=context.bot_data[poll_id]["chat_id"],
                        text=result,
                    )
                    break
                if cnt == 10:
                    context.bot.send_message(
                        chat_id=context.bot_data[poll_id]["chat_id"],
                        text=result,
                    )
        
        context.bot.send_message(
            chat_id=context.bot_data[poll_id]["chat_id"],
            text="You got all the recommendations!"
        )
        
        print("The End.")
        return

    message = context.bot.send_poll(
        context.bot_data[poll_id]["chat_id"],
        context.bot_data[context.bot_data[poll_id]['arr_q'][num_calls]][0][0], 
        context.bot_data[context.bot_data[poll_id]['arr_q'][num_calls]][1], 
        is_anonymous=False,
        allows_multiple_answers=False,
    )
    
    print(context.bot_data[poll_id]["chat_id"])
    
    payload = {
        message.poll.id: {
            "questions": survey_dict['q1'][1],
            "message_id": message.message_id,
            "chat_id": context.bot_data[poll_id]["chat_id"],
            "answers": 0,
            "count": num_calls + 1,
            "arr_q": context.bot_data[poll_id]["arr_q"],
            "array_of_answers": context.bot_data[poll_id]['array_of_answers'],
        }
    }
    context.bot_data.update(payload)

In [7]:
TOKEN = "YOUR TOKEN"
updater = Updater(token=TOKEN, use_context=True)
dispatcher = updater.dispatcher


dispatcher.add_handler(CommandHandler('start', start))
dispatcher.add_handler(CommandHandler('poll', poll))
dispatcher.add_handler(PollAnswerHandler(receive_poll_answer))

In [8]:
dispatcher.bot_data.update(survey_dict)

In [ ]:
updater.start_polling()

# Run the bot until the user presses Ctrl-C or the process receives SIGINT,
# SIGTERM or SIGABRT
updater.idle()